In [1]:
import tensorflow as tf
import keras
import pathlib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

2024-04-17 10:42:01.254908: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 10:42:01.254950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 10:42:01.256516: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-17 10:42:01.264690: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 10:42:03.561526: W tensorflow/compiler/tf2

In [2]:
model = keras.models.load_model('./first_model.h5')

2024-04-17 10:43:08.381379: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10230 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1
2024-04-17 10:43:08.382161: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10486 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2024-04-17 10:43:08.382901: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 10527 MB memory:  -> device: 2, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:81:00.0, compute capability: 6.1


In [3]:
tflite_models_dir = pathlib.Path("./tflite_models")
tflite_model_file = tflite_models_dir/"first_model.tflite"
tflite_model_quant_file = tflite_models_dir/"first_model_quant.tflite"


In [37]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


tflite_models_dir = pathlib.Path("./tflite_models")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

tflite_model_file = tflite_models_dir/"first_model.tflite"
tflite_model_file.write_bytes(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmppezt2hv1/assets


INFO:tensorflow:Assets written to: /tmp/tmppezt2hv1/assets


KeyboardInterrupt: 

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quant = converter.convert()

tflite_model_quant_file = tflite_models_dir/"first_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

INFO:tensorflow:Assets written to: /tmp/tmp9loz5pxd/assets


INFO:tensorflow:Assets written to: /tmp/tmp9loz5pxd/assets
2024-04-17 10:04:24.980631: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-04-17 10:04:24.980669: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-04-17 10:04:24.980921: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp9loz5pxd
2024-04-17 10:04:25.051447: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-17 10:04:25.051491: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp9loz5pxd
2024-04-17 10:04:25.205378: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-04-17 10:04:25.953337: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp9loz5pxd
2024-04-17 10:04:26.265049: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

4547632

In [4]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 8
test_dir = './converted_dataset/test'

In [10]:
global test_images, test_labels

test_dataset = keras.utils.image_dataset_from_directory(
    test_dir,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    seed=2024
)

subset = test_dataset.take(10)

test_images = []
test_labels = []
for images, labels in subset:
    for image in images:
        test_images.append(image)
    for label in labels:
        test_labels.append(label)
test_images = np.array(test_images)
test_labels = np.array(test_labels)


Found 14564 files belonging to 4 classes.


In [11]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
  global test_images

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_images[test_image_index]
    test_label = test_labels[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

  return predictions

In [12]:
# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_images
  global test_labels

  test_image_indices = range(test_images.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_labels== predictions) * 100) / len(test_images)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_images)))

In [14]:
evaluate_model(tflite_model_file, model_type="Float")

Float model accuracy is 100.0000% (Number of test samples=80)


In [15]:
evaluate_model(tflite_model_quant_file, model_type="Float")

Float model accuracy is 100.0000% (Number of test samples=80)
